In [2]:
%matplotlib inline

import os 
import json 

import boto3
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

COLOR_PALETTE = ['#648FFF', '#785EF0', '#DC267F', '#FE6100', '#FFB000']

In [8]:
s3_client = boto3.client('s3')
bucket = 'mimic-log-odds-models'
version = 2
prefix = f'movement-model-m4-a1-v{version}'

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)

configs = {}
if 'Contents' in response:
    for obj in response['Contents']:
        if obj['Key'].endswith('config.json'):
            config = json.loads(s3_client.get_object(Bucket=bucket, Key=obj['Key'])['Body'].read().decode('utf-8'))
            if 'model' in config:
                configs[config['run_id']] = config['model']
else:
    print("No objects found.")

rows = []
for run_id, config in configs.items():
    if 'num_layers' in config:
        rows.append({
            'run_id': run_id,
            'num_layers': config['num_layers'],
            'neurons': int(config['layers'][0][1:]),
            'dropout': 0.0 if not config['layers'][1].startswith('Dropout') else float(config['layers'][1][-1])/10
        })

configs_data = pd.DataFrame(rows)

results = (
    read_data_w_cache(f'select * from movement_model_experiment_m4_a1_v{version}')
    .sort_values(['run_id', 'epoch'], ascending=True)
)
results = results[['run_id', 'loss', 'val_loss', 'epoch']].merge(configs_data)
results.head()

,run_id,loss,val_loss,epoch,num_layers,neurons,dropout
0,09144850e0ae58aaf259179be959c19c9b4d60e54567c9...,0.625822,0.559701,1,3,16,0.1
1,09144850e0ae58aaf259179be959c19c9b4d60e54567c9...,0.485746,0.377522,2,3,16,0.1
2,09144850e0ae58aaf259179be959c19c9b4d60e54567c9...,0.380405,0.294497,3,3,16,0.1
3,09144850e0ae58aaf259179be959c19c9b4d60e54567c9...,0.336402,0.267655,4,3,16,0.1
4,09144850e0ae58aaf259179be959c19c9b4d60e54567c9...,0.309022,0.244122,5,3,16,0.1


In [9]:
final = results.groupby('run_id')[['val_loss', 'loss', 'epoch', 'neurons', 'dropout', 'num_layers']].last().reset_index().sort_values('val_loss', ascending=True)
final.head()

,run_id,val_loss,loss,epoch,neurons,dropout,num_layers
42,afb6299bb1ea4101cb01974b1fe52160d50ed48f534609...,0.208990,0.213220,100,16,0.1,4
62,ea9ee18c4f95ba60b4c78d2d5ba7f8350d9d53450d67f3...,0.211117,0.209551,100,24,0.2,3
11,3a5a052ba38dc309222c6cd4c40fbb6fb6a6bbe943e3f6...,0.213062,0.216380,100,24,0.2,3
50,c2ff76afdb233be4eb452c65474eb663f252d2e16a7c7d...,0.214561,0.213873,100,16,0.1,3
29,85cdb71e2ad3efa3c24221cd0e2071f761f8784582c8d0...,0.214759,0.214248,100,24,0.2,4


In [10]:
df = final.groupby(['neurons', 'dropout', 'num_layers'])[['val_loss', 'loss']].min().reset_index()
df['val_loss'].max() - df['val_loss'].min()

0.07811343669891357

In [11]:
run_id = final['run_id'].values[0]
run_id

'afb6299bb1ea4101cb01974b1fe52160d50ed48f53460923cb0432a7569200fa'

In [12]:
px.line(
    results[results['run_id'] == run_id], x='epoch', y='val_loss'
)